## Phase 4: Council valuation dataset
### INPUTS
Council Ratings Valuation (RV) data:
-	xls file for 2011 RVs
-	xls file for 2014 RVs
-	xls file for 2017 RVs
-	xls file for 2021 RVs [to be received later in the year]

### OUTPUTS
Four csv files (one for 2011, 2014, 2017 and 2021) with the following data fields added:
-	The same data fields listed through points 1-9 above under Phase 0 are to be added to each ratings valuation after each RV is matched to a parcel.
o	Naming convention: Use the same labels as given under Phase 1 (in red).


In [1]:
#pragma nodebook off
#Use nodebook for better reproducibility https://github.com/uoa-eResearch/nodebook
%reload_ext nodebook.ipython
%nodebook disk phase4

<IPython.core.display.Javascript object>

In [ ]:
# load libraries
import geopandas as gpd # vector data
import pandas as pd # tabular data, loading CSVs
import numpy as np # numeric data
from util import *
import matplotlib # plotting
import contextily as ctx # Used for contextual basemaps
from matplotlib_scalebar.scalebar import ScaleBar # scalebar for plot
import matplotlib.pyplot as plt # plotting
from tqdm.auto import tqdm # progress bars
tqdm.pandas()
import json
from scipy.spatial import cKDTree
from shapely.geometry import Point, shape, LineString, MultiLineString, GeometryCollection, MultiPoint, Polygon  # creating points
plt.rcParams['figure.figsize'] = (20, 20)
pd.set_option('max_columns', None)

In [ ]:
%%time
rvs = {}
for year in tqdm([2011, 2014, 2017]):
    rvs[year] = pd.read_excel(f"restricted/REVAL{year} Property Data Supply.xlsx")

  0%|          | 0/3 [00:00<?, ?it/s]

CPU times: user 8min 56s, sys: 14.2 s, total: 9min 11s
Wall time: 9min 11s


10.	Because titles can be cumbersome to work with, please add a unique numeric identifier for each RV in the dataset based on its certificate of title. Call it ‘FID_certificate_of_title’. 

In [ ]:
rvs[2017]

,Valuation_Number_Roll,Valuation_Number_Assessment,Valuation_Number_Suffix,District_Code,Situation_Number,Additional_Situation_Number,Situation_Name,Legal_Description,Land_Area,Property_Category,Ownership_Code,Current_Effective_Valuation_Date,Capital_Value,Improvements_Value,Land_Value,Trees,REVISED_CAPITAL_VALUE,REVISED_IMPROVEMENTS_VALUE,REVISED_LAND_VALUE,REVISED_TREES,Improvements_Description,Certificate_of_Title,Additional_Certificate_of_Title,Zoning,Actual_Property_Use,Units_of_Use,Off_street_Parking,Building_Age_Indicator,Building_Condition_Indicator,Building_Construction_Indicator,Building_Site_Coverage,Building_Total_Floor_Area,Mass_Contour,Mass_View,Mass_Scope_of_View,Mass_Total_Living_Area,Mass_Deck,Mass_Workshop_Laundry,Mass_Other_Improvements,Mass_Garage_Freestanding,Mass_Garage_Under_Main_Roof,Sales_Group
0,38703,44617,NaN,14,84.0,NaN,REYNOLDS RD,LOT 17 DP344916,0.0836,RD201B,1.0,2014-07-01,860000,450000,410000,NaN,0,0,0,0,DWG GGE OI,184249,N,9B,91,1.0,3.0,201,GG,BT,229.0,229.0,ER,O,S,173.0,N,N,N,NaN,3.0,1002
1,38703,44618,NaN,14,82.0,NaN,REYNOLDS RD,LOT 18 DP344916,0.0829,RD200B,1.0,2014-07-01,820000,410000,410000,NaN,0,0,0,0,DWG OI,184250,N,9B,91,1.0,NaN,200,GG,BT,222.0,222.0,ER,O,M,184.0,N,N,N,NaN,2.0,1002
2,38703,44619,NaN,14,80.0,NaN,REYNOLDS RD,LOT 19 DP344916,0.0839,RD200B,1.0,2014-07-01,820000,410000,410000,NaN,0,0,0,0,DWG OI,184251,N,9B,91,1.0,NaN,200,GG,BT,212.0,212.0,ER,O,S,172.0,N,N,N,NaN,2.0,1002
3,38703,44621,NaN,14,22.0,NaN,REYNOLDS RD,LOT 21 DP344916,5.5664,OS,4.0,2014-07-01,1080000,5000,1075000,NaN,0,0,0,0,OB,184253,N,5A,55,1.0,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1002
4,38703,44622,NaN,14,NaN,NaN,REYNOLDS RD,LOT 22 DP344916,1.7965,OP,4.0,2014-07-01,500000,0,500000,NaN,0,0,0,0,NaN,184254,N,5B,55,1.0,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1002
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
549931,2390,50700,B,14,33.0,A,Ellerton Road,"Flat 2 DP 114220, Lot 1 DP 111459 938m2",0.0000,RD198B,1.0,2014-07-01,1475000,275000,1200000,0.0,1475000,275000,1200000,0,HOUSE,65A/204,N,9B,91,1.0,1.0,198,GG,BT,110.0,172.0,LV,N,N,120.0,Y,N,N,0.0,1.0,730
549932,2400,33400,NaN,14,84.0,NaN,Peary Road,Lot 54 DP 20380 518m2,0.0518,RD192B,1.0,2014-07-01,1360000,140000,1220000,0.0,1360000,140000,1220000,0,HOUSE,454/190,N,9A,91,1.0,0.0,192,FF,WI,120.0,119.0,LV,N,N,119.0,N,N,N,0.0,0.0,730
549933,2400,34500,NaN,14,106.0,NaN,Peary Road,Lot 43 DP 17805 822m2,0.0822,RD191B,1.0,2014-07-01,2000000,550000,1450000,0.0,2000000,550000,1450000,0,HOUSE & GARAGE,82C/174,N,9A,91,1.0,1.0,191,AA,WI,190.0,260.0,LV,N,N,260.0,Y,N,N,1.0,0.0,730
549934,2400,34900,NaN,14,114.0,NaN,Peary Road,Lot 39 DP 17805 541m2,0.0541,RD192A,1.0,2014-07-01,1650000,410000,1240000,0.0,1650000,410000,1240000,0,HOUSE & GARAGE,413/13,N,9A,91,1.0,2.0,192,AA,WI,140.0,145.0,LV,N,N,145.0,N,N,N,2.0,0.0,730


In [ ]:
for rv in rvs.values():
    rv.Certificate_of_Title = rv.Certificate_of_Title.astype(str)

In [ ]:
len(rvs[2017].Certificate_of_Title), len(rvs[2017].Certificate_of_Title.unique())

(549936, 543313)

In [ ]:
all_cots = pd.concat(rv.Certificate_of_Title for rv in rvs.values()).unique()
all_cots.sort()
print(len(all_cots))
all_cots = {cot: i for i, cot in enumerate(all_cots)}

740915


In [ ]:
%%time
for rv in rvs.values():
    rv["FID_certificate_of_title"] = rv.Certificate_of_Title.progress_apply(lambda cot: all_cots[cot])

  0%|          | 0/516469 [00:00<?, ?it/s]

  0%|          | 0/529046 [00:00<?, ?it/s]

  0%|          | 0/549936 [00:00<?, ?it/s]

CPU times: user 8.5 s, sys: 2.14 s, total: 10.6 s
Wall time: 11.1 s


PROCEDURE FOR MATCHING RVs TO LINZ DATABASE  
Use certificate of title to match each valuation to a LINZ parcel(s). 

11.	Indicator for one-to-many matched LINZ parcels: ‘One_to_many_matched_parcels’  
a.	Equal to one or zero

In [ ]:
rv = rvs[2017]

In [ ]:
%%time
phase0 = pd.read_csv("output/parcels_phase0.csv", low_memory=False, lineterminator="\n")
phase0

CPU times: user 12.5 s, sys: 7.36 s, total: 19.8 s
Wall time: 20.1 s


,LINZ_parcel_ID,appellation,affected_surveys,parcel_intent,topology_type,statutory_actions,land_district,titles,survey_area,calc_area,LINZ_parcel_centroid_lon,LINZ_parcel_centroid_lat,LINZ_parcel_vertices_lon,LINZ_parcel_vertices_lat,LINZ_parcel_roadvertices_lon,LINZ_parcel_roadvertices_lat,LINZmatch_AUP_code,LINZmatch_AUP_name,LINZ_adjoining_parcel_ID,LINZ_parcel_sides_zones,LINZ_TRNSPWR_ohead_name,LINZ_TRNSPWR_ohead_indicator,LINZ_VWSHFT_ohead_name,LINZ_VWSHFT_ohead_ID,LINZ_VWSHFT_ohead_indicator,Hdist_rural,Hdist_rural_code,Hdist_rural_name,Hdist_bus,Hdist_bus_code,Hdist_bus_name,Hdist_resid,Hdist_resid_code,Hdist_resid_name,Hdist_SH,Hdist_MHS,Hdist_MHU,Hdist_THA,Local_Area_name,SA22018_name,SA22018_code,AU2013_name,AU2013_code,MB2018_code,MB2013_code,Hdist_coast,Hdist_motorway,Hdist_main_road,Hdist_rail,Hdist_skytower,SpHA_indicator
0,4918991,Lot 327 DP 56341,DP 56341,DCDB,Primary,NaN,North Auckland,NA8C/911,642.0,643.0,174.843218,-36.876042,[174.842994; 174.843152; 174.84337; 174.843471...,[-36.875922; -36.875957; -36.876006; -36.87602...,[174.842994; 174.842997],[-36.875922; -36.876074],18.0,Residential - Mixed Housing Suburban Zone,"[5061331, 5002265, 5228435, 5061335, 4722548, ...","['18', '18', '27', '18', '18', '18']",NaN,0,NaN,NaN,0,7228.891309,3,Rural - Countryside Living Zone,96.953993,44,Business - Neighbourhood Centre Zone,0.000000,18,Residential - Mixed Housing Suburban Zone,1499.258244,0.000000,463.752596,342.351869,Orakei,Saint Johns East,144500.0,St Johns,517201,4006757.0,465102,1626.866018,3904.389673,30.431054,675.367028,7850.889144,False
1,4919914,Lot 178 DP 56342,DP 56342,DCDB,Primary,NaN,North Auckland,"474924, NA8C/768",650.0,651.0,174.846572,-36.873551,[174.84645; 174.846486; 174.846701; 174.846648],[-36.873699; -36.873386; -36.873407; -36.873721],[174.846486; 174.846701],[-36.873386; -36.873407],18.0,Residential - Mixed Housing Suburban Zone,"[4789870, 4877424, 5005927, 4804873, 5178022, ...","['18', '18', '18', '18', '18', '27']",NaN,0,NaN,NaN,0,7571.289947,3,Rural - Countryside Living Zone,219.321245,17,Business - Light Industry Zone,0.000000,18,Residential - Mixed Housing Suburban Zone,1504.842901,0.000000,380.053451,411.291837,Orakei,Saint Johns East,144500.0,St Johns,517201,4006774.0,464800,1666.931163,4307.322450,275.948865,296.455456,8026.797757,False
2,4926734,Lot 144 DP 53562,DP 53562,DCDB,Primary,NaN,North Auckland,"386353, 386354, 386355, 386356, NA4D/859",640.0,640.0,174.845299,-36.873269,[174.845069; 174.845157; 174.845513; 174.84546],[-36.87326; -36.873123; -36.87327; -36.873421],[174.845069; 174.845157],[-36.87326; -36.873123],18.0,Residential - Mixed Housing Suburban Zone,"[5236861, 5048379, 4926945, 4797737]","['27', '18', '18', '18']",NaN,0,NaN,NaN,0,7574.099005,3,Rural - Countryside Living Zone,197.076711,44,Business - Neighbourhood Centre Zone,0.000000,18,Residential - Mixed Housing Suburban Zone,1551.382113,0.000000,336.510990,468.042305,Orakei,Saint Johns East,144500.0,St Johns,517201,4006774.0,464800,1556.501519,4232.755849,170.250936,317.577857,7909.576996,False
3,4926945,Lot 183 DP 56342,DP 56342,DCDB,Primary,NaN,North Auckland,"433646, NA8C/773",913.0,914.0,174.845585,-36.873350,[174.845775; 174.84557; 174.845408; 174.84546;...,[-36.873177; -36.873603; -36.87357; -36.873421...,[174.845775; 174.845575; 174.845689],[-36.873177; -36.873095; -36.873142],18.0,Residential - Mixed Housing Suburban Zone,"[5233326, 5048379, 4797737, 5054505, 7342193, ...","['27', '18', '18', '18', '18', '18', '18', '18']",NaN,0,NaN,NaN,0,7556.977156,3,Rural - Countryside Living Zone,180.285687,44,Business - Neighbourhood Centre Zone,0.000000,18,Residential - Mixed Housing Suburban Zone,1529.480918,0.000000,333.915495,445.926909,Orakei,Saint Johns East,144500.0,St Johns,517201,4006774.0,464800,1582.400173,4248.261840,195.278997,314.544307,7936.587996,False
4,4929034,Lot 260 DP 56341,DP 56341,DCDB,Primary,NaN,North Auckland,"488577, NA8C/850",622.0,624.0,174.847627,-36.874955,[174.847427; 174.847674; 1

In [ ]:
RV_NAs = sum(rv.Certificate_of_Title.str.contains('NA'))
phase0_NAs = sum(phase0.titles.str.contains('NA', na=False))
print(f"{RV_NAs} RV titles start with NA / {len(rv)} ({RV_NAs/len(rv)*100}%)")
print(f"{phase0_NAs} parcel titles start with NA / {len(phase0)} ({phase0_NAs/len(phase0)*100}%)")

322573 RV titles start with NA / 549936 (58.65646184283262%)
345003 parcel titles start with NA / 547902 (62.96801252778782%)


In [ ]:
print(sum(rv.Certificate_of_Title.isin(phase0.titles)))
print(sum(rv.Certificate_of_Title.str.replace("NA", "").isin(phase0.titles.str.replace("NA", ""))))

302219
351924


In [ ]:
rv.Certificate_of_Title = rv.Certificate_of_Title.str.replace("NA", "")
phase0.titles = phase0.titles.str.replace("NA","")

In [ ]:
%%time
def find_parcel_match(row):
    matches = phase0[phase0.titles.str.contains(row.Certificate_of_Title, na=False)]
    row["matched_parcels"] = len(matches)
    row["One_to_many_matched_parcels"] = len(matches) > 1
    if len(matches):
        return pd.concat([row, matches.iloc[0]], axis=1)
    else:
        return row
rv = rv.progress_apply(find_parcel_match, axis=1)
rv

  0%|          | 0/549936 [00:00<?, ?it/s]

12.	Indicator for many-to-one matched LINZ parcel: ‘Many_to_one_matched_parcels’  
a.	Equal to one or zero  